In [1]:
from bs4 import BeautifulSoup as bs
import random
import time
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC

In [3]:
data_urls = pd.read_csv('urls_trovit.csv')  ## datos de urls

In [4]:
data_urls.shape

(475, 2)

In [12]:
browser = uc.Chrome(driver_executable_path='../chromedriver-linux64/chromedriver') ## iniciamos el navegador

In [4]:
path = 'https://casas.trovit.com.co/listing/'
url =  path+data_urls.iloc[0,0]
url

'https://casas.trovit.com.co/listing/vendo-apartamento-cerca-a-unicentro-tunja.04b11625-6459-4b81-9ed0-5b1cca149c97'

In [152]:
browser.get(url)

In [153]:
html = browser.page_source ## sacamos el html de la pagina

In [154]:
soup = bs(html,"html")

In [155]:
temp = soup.find('div',{'id','amenities'}).find_all('li')
li = [i for i in temp]

In [179]:
valor = [data_urls.iloc[0,0]]
clave = ['url']

for i in li:
    clave.append(i.find('div',{'class','amenity-key'}).text.strip())
    valor.append(i.find('div',{'class','amenity-value'}).text.strip())

In [180]:
list(zip(clave,valor))

[('url',
  'vendo-apartamento-cerca-a-unicentro-tunja.04b11625-6459-4b81-9ed0-5b1cca149c97'),
 ('Tipo de anuncio', 'Venta'),
 ('Tipo de propiedad', 'Apartamento'),
 ('Habitaciones', '3'),
 ('Baños', '2'),
 ('Superficie', '110m²'),
 ('Área construida', '110m²'),
 ('Piso', '3'),
 ('Estrato', '5')]

In [181]:
price = soup.find('div',{'class','price'}).find('div',{'class','amount'}).text.strip()
patron = re.compile(r'\$\s?([\d\.]+)')

# Buscar la coincidencia
coincidencia = patron.search(price)

# Extraer el texto encontrado
if coincidencia:
    precio = coincidencia.group(1)
    clave.append('precio')
    valor.append(precio)

In [182]:
map = soup.find('div',{'class','card js-map-location'})
    
clave.append('latitud')
valor.append(map.find('div',{'class':'location_map'}).get('data-latitude'))
clave.append('longitud')
valor.append(map.find('div',{'class':'location_map'}).get('data-longitude'))

In [178]:
map.find('div',{'class':'location_map'}).get('data-latitude')

'5,5465998'

In [183]:
ubicacion = soup.find('div',{'class','location_address'})
clave.append('ubicacion')
valor.append(ubicacion.text)


In [184]:
data = pd.DataFrame([valor],columns=clave)
data

,url,Tipo de anuncio,Tipo de propiedad,Habitaciones,Baños,Superficie,Área construida,Piso,Estrato,precio,latitud,longitud,ubicacion
0,vendo-apartamento-cerca-a-unicentro-tunja.04b1...,Venta,Apartamento,3,2,110m²,110m²,3,5,460.000.000,"5,5465998","-73,3472673","Conjunto Residencial Mirador De La Colina, T..."


In [33]:
data = pd.concat([data,df],ignore_index=True)

In [5]:
data = pd.read_csv('data_trovit.csv')

## Automatizando para 475

In [103]:
path = 'https://casas.trovit.com.co/listing/'
for i in range(25,data_urls.shape[0]):
    url =  path+data_urls.iloc[i,0]
    browser.get(url)
    time.sleep(random.randint(10,12))
    html = browser.page_source ## sacamos el html de la pagina
    soup = bs(html,"html")
    temp = soup.find('div',{'id','amenities'}).find_all('li')
    li = [i for i in temp]
    valor = []
    clave = []
    for i in li:
        clave.append(i.find('div',{'class','amenity-key'}).text.strip())
        valor.append(i.find('div',{'class','amenity-value'}).text.strip())
    ## precio
    price = soup.find('div',{'class','price'}).find('div',{'class','amount'}).text.strip()
    patron = re.compile(r'\$\s?([\d\.]+)')

    # Buscar la coincidencia
    coincidencia = patron.search(price)

    # Extraer el texto encontrado
    if coincidencia:
        precio = coincidencia.group(1)
        clave.append('precio')
        valor.append(precio)
    # coodenadas geograficas
    time.sleep(random.randint(6,7))
    ## volvemos a scrapear la pagina 
    html = browser.page_source ## sacamos el html de la pagina
    soup = bs(html,"html")
    map = soup.find('div',{'class','location_map static-image'})
    clave.append('latitud')
    try:
        valor.append(map.get('data-latitude'))
    except NoneType:
        browser.refresh()
        html = browser.page_source ## sacamos el html de la pagina
        soup = bs(html,"html")
        map = soup.find('div',{'class','card js-map-location'})
        valor.append(map.get('data-latitude'))
    clave.append('longitud')
    valor.append(map.get('data-longitude'))
    # texto de ubicacion
    ubicacion = soup.find('div',{'class','location_address'})
    clave.append('ubicacion')
    valor.append(ubicacion.text)
    df = pd.DataFrame([valor],columns=clave)
    data = pd.concat([data,df],ignore_index=True)

NameError: name 'NoneType' is not defined

In [6]:
data.shape

(324, 18)

In [13]:
path = 'https://casas.trovit.com.co/listing/'
for j in range(395,data_urls.shape[0]):
    url =  path+data_urls.iloc[j,0]
    browser.get(url)
    time.sleep(random.randint(10,12))
    html = browser.page_source ## sacamos el html de la pagina
    soup = bs(html,"html")
    try:
        temp = soup.find('div',{'id','amenities'}).find_all('li')
    except AttributeError:
        print('no se encontro')
        continue
    li = [i for i in temp]
    valor = [data_urls.iloc[j,0]]
    clave = ['url']
    for i in li:
        clave.append(i.find('div',{'class','amenity-key'}).text.strip())
        valor.append(i.find('div',{'class','amenity-value'}).text.strip())
    ## precio
    dec = True
    try:
        price = soup.find('div',{'class','price'}).find('div',{'class','amount'}).text.strip()
    except AttributeError:
        price = soup.find('div',{'class','price'}).find('span',{'class','amount'})
        clave.append('precio')
        if price != None:
            valor.append(price.text)
        else:
            valor.append(np.nan)
    
        dec = False
  
    if dec:
        patron = re.compile(r'\$\s?([\d\.]+)')
        coincidencia = patron.search(price)

            # Extraer el texto encontrado
        if coincidencia:
            precio = coincidencia.group(1)
            clave.append('precio')
            valor.append(precio)
    
    # coodenadas geograficas
    time.sleep(random.randint(5,6))
    ## volvemos a scrapear la pagina 
    html = browser.page_source ## sacamos el html de la pagina
    soup = bs(html,"html")
    map = soup.find('div',{'class','card js-map-location'})
    if map != None:
        clave.append('latitud')
        valor.append(map.find('div',{'class':'location_map'}).get('data-latitude'))
        clave.append('longitud')
        valor.append(map.find('div',{'class':'location_map'}).get('data-longitude'))
    else:
        clave.append('latitud')
        valor.append(np.nan)
        clave.append('longitud')
        valor.append(np.nan)
    
    '''
    
    try:
        valor.append(l[5].get('data-latitude'))
    except NoneType:
        browser.refresh()
        html = browser.page_source ## sacamos el html de la pagina
        soup = bs(html,"html")
        map = soup.find('div',{'class','card js-map-location'})
        valor.append(map.get('data-latitude'))
    clave.append('longitud')
    valor.append(l[5].get('data-longitude'))
    '''
    # texto de ubicacion
    ubicacion = soup.find('div',{'class','location_address'})
    if ubicacion != None:
        clave.append('ubicacion')
        valor.append(ubicacion.text)
    else:
        clave.append('ubicacion')
        valor.append(np.nan)
    df = pd.DataFrame([valor],columns=clave)
    data = pd.concat([data,df],ignore_index=True)

no se encontro
no se encontro
no se encontro
no se encontro


In [25]:
url

'https://casas.trovit.com.co/listing/apartamento-en-venta-en-tunja-v113102.1S15sGL1Lj1z'

In [14]:
j

474

In [15]:
 data.tail()

,url,Tipo de anuncio,Tipo de propiedad,Habitaciones,Baños,Superficie,Área construida,Piso,Estrato,precio,latitud,longitud,ubicacion,Año de construcción,Baño de visitas,Superficie del terreno,Parqueadero,Orientación
455,apartamento-en-venta.1Ng6I12b_b,Venta,Apartamento en Venta,2,2,52m²,NaN,NaN,NaN,$217M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,"apartamento-en-venta,-la-maria.L9Eoxk1Q1H",Venta,Apartamento,1,1,32m²,NaN,NaN,NaN,$110M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,venta-apartamento-ramiriqui-boyaca.R1D1-o12w1f_,Venta,Apartamento,4,3,81m²,NaN,NaN,NaN,$360M,NaN,NaN,NaN,NaN,NaN,NaN,Sí,NaN
458,lote-en-zona-de-expancion-al-norte-de-tunja.C1...,Venta,Terreno o Lote,NaN,NaN,6.225m²,NaN,NaN,NaN,$1.250M,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN
459,apartamento-en-venta.O1AVejd1V18,Venta,Apartamento en Venta,2,1,69m²,NaN,NaN,NaN,$199M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data.to_csv('data_trovit.csv',index=False)

In [94]:
l[5].get("data-longitude")

'-73,3575572'

In [24]:
path+data_urls.iloc[j,0]

'https://casas.trovit.com.co/listing/apartamento-en-venta-en-tunja-v113102.1S15sGL1Lj1z'

In [109]:
i

24

In [7]:
a = 1
b = [1,3,0,6,4]
for i in b:
    try:
        print(a/i)
    except ZeroDivisionError:
        continue

1.0
0.3333333333333333
0.16666666666666666
0.25


In [25]:
data.to_csv('data_trovit.csv',index =False)